<a href="https://colab.research.google.com/github/shiny880410/helloworld/blob/master/hw1/Frechet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gspread
!pip install oauth2client
!pip install Pillow
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 20.0MB/s 


In [5]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': "'1_7FyladR7eGnmUjsCvNF1XnpVZpuNGXm' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  #fname = os.path.join(local_download_path, f['title'])
  fname = os.path.join('/content', f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
!ls "/root/data"

title: Frechet.ipynb, id: 1rjh78sM8vRXoyqdOgDt7YAYgchjawvHG
downloading to /content/Frechet.ipynb
title: 標記顯著速度差異.ipynb, id: 1AM9X_LwPT7wA4lDu42OQ-tbc23q_vedD
downloading to /content/標記顯著速度差異.ipynb
title: 復興南路2.ipynb, id: 1QyxaBmyotcR5YhBxm7rtT7xlnma5byNr
downloading to /content/復興南路2.ipynb
title: NotoSansCJKtc-Regular.otf, id: 1fY1VCoDLMTrMyEQLrXghet2ZJjKX3iZv
downloading to /content/NotoSansCJKtc-Regular.otf


In [8]:
def euc_dist(pt1,pt2):
    return math.sqrt((pt2[0]-pt1[0])*(pt2[0]-pt1[0])+(pt2[1]-pt1[1])*(pt2[1]-pt1[1]))

def _c(ca,i,j,P,Q):
    if ca[i,j] > -1:
        return ca[i,j]
    elif i == 0 and j == 0:
        ca[i,j] = euc_dist(P[0],Q[0])
    elif i > 0 and j == 0:
        ca[i,j] = max(_c(ca,i-1,0,P,Q),euc_dist(P[i],Q[0]))
    elif i == 0 and j > 0:
        ca[i,j] = max(_c(ca,0,j-1,P,Q),euc_dist(P[0],Q[j]))
    elif i > 0 and j > 0:
        ca[i,j] = max(min(_c(ca,i-1,j,P,Q),_c(ca,i-1,j-1,P,Q),_c(ca,i,j-1,P,Q)),euc_dist(P[i],Q[j]))
    else:
        ca[i,j] = float("inf")
    return ca[i,j]

def frechetDist(P,Q):
    ca = np.ones((len(P),len(Q)))
    ca = np.multiply(ca,-1)
    return _c(ca,len(P)-1,len(Q)-1,P,Q)
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import animation, rc
from IPython.display import HTML
from IPython.display import Image
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
import io
import math

gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1FJPf9S2vpimDZvefrpnfq31cq3JpmySHse74WQoEgu4/edit?ts=5c8895e5#gid=618695640')
wks4 = sh.worksheet("復興南北路")


for w in [21,22,23,24,25,26]:
    qp = wks4.col_values(w)
    yp = wks4.col_values(20)
    yp.pop(0)
    yp.pop(0)
    yp.pop(0)
    qp.pop(0)
    qp.pop(0)
    qp.pop(0)
    x=[]
    P=[]
    Q=[]
    for i in range(len(yp)):
        yp[i]=eval(yp[i])
        qp[i]=eval(qp[i])
        x.append(10*(i+1))
    for j in range(len(yp)):
        P.append([x[j],yp[j]])
        Q.append([x[j],qp[j]])

    a=frechetDist(P,Q)
    print(a)


2.1373744812
2.6625679680000003
2.880447385
3.004080398
3.0921094179999997
2.5273821506000003
